B1

In [3]:
import sys
#!{sys.executable} -m pip install numpy
!{sys.executable} -m nltk.downloader punkt


[nltk_data] Downloading package punkt to /Users/ghz/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
import sys
reload(sys)
sys.setdefaultencoding('utf8')

In [1]:
import numpy as np
import pandas as pd
import re
from pandas import Series, DataFrame
from nltk import word_tokenize
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
#raw = pd.read_csv('Train_rev1.csv')

In [156]:
mask1=raw['ContractType'].notnull()
mask2=raw['ContractTime'].notnull()

mask3=(mask1&mask2)
mask4=raw['SalaryNormalized'].notnull()
mask5=(mask3&mask4)
df_subset_temp=raw[mask5]
df_subset=df_subset_temp.sample(n=2500)



In [3]:
#df_subset.to_csv('df_subset.csv',header=True,sep=',')
df_cleaned = pd.read_csv('df_subset.csv')

In [4]:
df_cleaned['Salary']=0
df_cleaned['Salary'][df_cleaned['SalaryNormalized']>=40000]=1.0
df_cleaned["ContractType_dm"]=np.where(df_cleaned["ContractType"]=="full_time",0,1)
df_cleaned["ContractTime_dm"]=np.where(df_cleaned["ContractTime"]=="contract",0,1)
df_cleaned["LocationNormalized_dm"]=np.where(df_cleaned["LocationNormalized"]=="Central London",0,
                                            np.where(df_cleaned["LocationNormalized"]=='London',0,
                                                    np.where(df_cleaned["LocationNormalized"]=='South East London',0,1)))

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [160]:
dummies=pd.get_dummies(df_cleaned['Category'],prefix=None,prefix_sep="_",dummy_na=False,columns=None,drop_first=False)


df_use=df_cleaned[['ContractType_dm','ContractType_dm','LocationNormalized_dm','Salary']]
df_cal=pd.merge(df_use,dummies,left_on=df_use.index,right_on=dummies.index)
df_cal2=df_cal.drop(['Salary'],axis=1)
df_x=df_cal2.set_index('key_0')
df_y=df_cal.set_index('key_0')

In [161]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_x, df_y['Salary'], test_size=0.3, random_state=1)

In [162]:
gnb = GaussianNB()

In [163]:
gnb.fit(
   X_train, y_train
)
y_pred = gnb.predict(X_test)


In [164]:
print "Number of mislabeled points out of a total {} points : {}, performance {:05.2f}%".format(
          X_test.shape[0],
          (y_test != y_pred).sum(),
          100*(1-float((y_test != y_pred).sum())/float(X_test.shape[0]))
)

In [165]:
float((y_test != y_pred).sum())/float(X_test.shape[0])


0.6306666666666667

B2

In [7]:
from nltk.tokenize import word_tokenize
df_cleaned['description_low']=df_cleaned['FullDescription'].map(lambda x: x.lower())

m=df_cleaned['description_low'].tolist()
all_words=''.join(m)
all_words_list=word_tokenize(all_words)
all_words_list



def s(t):
    return word_tokenize(t)
a=df_cleaned['description_low'].map(s)



df_cleaned['Salary']=0
df_cleaned['Salary'][df_cleaned['SalaryNormalized']>=40000]=1.0



from sklearn.model_selection import train_test_split

X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(df_cleaned['description_low'], df_cleaned['Salary'], 
                                                            test_size=0.3, random_state=1)



In [8]:
subset_train=pd.concat([X_train_b,y_train_b],axis=1) 
listofeach_train=subset_train.values.tolist()
t_train= [({word: (word in word_tokenize(x[0])) for word in all_words_list}, x[1]) for x in listofeach_train]

KeyboardInterrupt: 

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(t_train)

In [ ]:
subset_test=pd.concat([X_test_b,y_test_b],axis=1)
listofeach_test=subset_test.values.tolist()
t_test= [({word: (word in word_tokenize(x[0])) for word in all_words_list}, x[1]) for x in listofeach_test]

In [ ]:
classifier.classify(t_test)

In [ ]:
float((y_test_b!=classifier.classify(t_test)).sum())/float(y_test_b.shape[0])